**Workflow**:

---

1. Current Recharge Trending - Break into Yearly, Quarterly, Weekly Seasonality, plus Trend
2. Optimized Forecasting model - forecast for the next 7 days
3. daily update workflow
4. Final output - Recharge forecast Vs. Actual, Additives, Error accuracy.

In [ ]:
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd



In [ ]:
df = pd.read_excel('/content/Ufone.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

m = Prophet(daily_seasonality=False)
m.add_seasonality(name='weekly', period=7, fourier_order=3)
m.add_seasonality(name='monthly', period=30, fourier_order=6)
m.add_seasonality(name='quarterly', period=90, fourier_order=6)
m.add_seasonality(name='annual', period=365, fourier_order=10)
m.add_country_holidays(country_name='PAK')

m.fit(df)

df_future = m.make_future_dataframe(periods=30)
forecast = m.predict(df_future)


INFO:prophet:Found custom seasonality named 'weekly', disabling built-in 'weekly' seasonality.


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '365 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

INFO:prophet:Making 21 forecasts with cutoffs between 2024-01-28 00:00:00 and 2025-09-19 00:00:00


  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive

forecast.to_excel('/content/forecast.xlsx', index=False)

In [ ]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

In [ ]:
%cd /content
!rm -rf glowing-adventure

/content
